<a href="https://colab.research.google.com/github/sashreek-das/starcoder2/blob/main/GO_Finetune_Model_StarCoder2_3b_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/bigcode-project/starcoder2.git

Cloning into 'starcoder2'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 44 (delta 19), reused 9 (delta 2), pack-reused 0
Receiving objects: 100% (44/44), 21.08 KiB | 353.00 KiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
!ls
%cd starcoder2

sample_data  starcoder2
/content/starcoder2


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate -U
!pip install datasets>=2.16.1
!pip install bitsandbytes==0.41.3
!pip install peft==0.8.2
!pip install trl==0.7.10
!pip install wandb==0.16.3
!pip install huggingface_hub==0.20.3

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-0r9bspu6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-0r9bspu6
  Resolved https://github.com/huggingface/transformers.git to commit fce52cefa744a5900fc065aafb2f55d846d1202c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8806756 sha256=5ceb263e6cca70251a718558916acea0853397691c29665406f0120436876a97
  Stored in directory: /tmp/pip-ephem-wheel-cache-cxiee4p0/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# to use 4bit use `load_in_4bit=True` instead
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

checkpoint = "bigcode/starcoder2-3b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, quantization_config=quantization_config)

inputs = tokenizer.encode("def print_hello_world():", return_tensors="pt").to("cuda")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1137: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


def print_hello_world():
    print("Hello World")

print_hello_world


In [ ]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e6:.2f} MB")

Memory footprint: 1994.90 MB


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
load_dataset("bigcode/the-stack-smol", data_dir="data/go")

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang'],
        num_rows: 10000
    })
})

In [ ]:
!pip install git+https://github.com/huggingface/trl.git
!pip install peft

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-boxkopdu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-boxkopdu
  Resolved https://github.com/huggingface/trl.git to commit ab0d11d81550618546cd1d8807627a594a58f029
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.8.2.dev0-py3-none-any.whl size=238379 sha256=d6bc5a83df3f9161e37630b6e1d812b4b73eaa5732e8ced6a3a6209e0c0fd4de
  Stored in directory: /tmp/pip-ephem-wheel-cache-tfoh_59v/wheels/22/0e/42/319b77b2648bb6140ef2b08b0478ede9ca3cc7879fcd022d36
Successfully built trl
  Attempting uninstall: trl
    Found existing installation: trl 0.7.10
    Uninstalling trl-0.7.10:
      Successfully uninstalled trl-0.7.10


In [ ]:
import argparse
import multiprocessing
import os

import torch
import transformers
from accelerate import PartialState
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    logging,
    set_seed,
)
from trl import SFTTrainer

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--model_id", type=str, default="bigcode/starcoder2-3b")
parser.add_argument("--dataset_name", type=str, default="bigcode/the-stack-smol")
parser.add_argument("--subset", type=str, default="data/go")
parser.add_argument("--split", type=str, default="train")
parser.add_argument("--dataset_text_field", type=str, default="content")

parser.add_argument("--max_seq_length", type=int, default=512)
parser.add_argument("--max_steps", type=int, default=1000)
parser.add_argument("--micro_batch_size", type=int, default=1)
parser.add_argument("--gradient_accumulation_steps", type=int, default=4)
parser.add_argument("--weight_decay", type=float, default=0.01)
parser.add_argument("--fp16", type=bool, default=True)

parser.add_argument("--attention_dropout", type=float, default=0.1)
parser.add_argument("--learning_rate", type=float, default=2e-4)
parser.add_argument("--lr_scheduler_type", type=str, default="cosine")
parser.add_argument("--warmup_steps", type=int, default=100)
parser.add_argument("--seed", type=int, default=0)
parser.add_argument("--output_dir", type=str, default="finetunedGO_starcoder2")
parser.add_argument("--num_proc", type=int, default=2)#T4 gpu of google colab
parser.add_argument("--push_to_hub", type=bool, default=True)

args = parser.parse_args(args=[])

In [ ]:
args

Namespace(model_id='bigcode/starcoder2-3b', dataset_name='bigcode/the-stack-smol', subset='data/go', split='train', dataset_text_field='content', max_seq_length=512, max_steps=1000, micro_batch_size=1, gradient_accumulation_steps=4, weight_decay=0.01, fp16=True, attention_dropout=0.1, learning_rate=0.0002, lr_scheduler_type='cosine', warmup_steps=100, seed=0, output_dir='finetunedGO_starcoder2', num_proc=2, push_to_hub=True)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
def main(args):
    # config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    lora_config = LoraConfig(
        r=8,
        target_modules=[
            "q_proj",
            "o_proj",
            "k_proj",
            "v_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        task_type="CAUSAL_LM",
    )

    # load model and dataset
    token = os.environ.get("HF_TOKEN", None)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_id,
        quantization_config=bnb_config,
        device_map={"": PartialState().process_index},
        attention_dropout=args.attention_dropout,
    )
    print_trainable_parameters(model)

    data = load_dataset(
        args.dataset_name,
        data_dir=args.subset,
        split=args.split,
        token=token,
        num_proc=args.num_proc if args.num_proc else multiprocessing.cpu_count(),
    )

    # setup the trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        max_seq_length=args.max_seq_length,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=args.micro_batch_size,
            gradient_accumulation_steps=args.gradient_accumulation_steps,
            warmup_steps=args.warmup_steps,
            max_steps=args.max_steps,
            learning_rate=args.learning_rate,
            lr_scheduler_type=args.lr_scheduler_type,
            weight_decay=args.weight_decay,
            fp16=args.fp16,
            logging_strategy="steps",
            logging_steps=10,
            output_dir=args.output_dir,
            optim="paged_adamw_8bit",
            seed=args.seed,
            run_name=f"train-{args.model_id.split('/')[-1]}",
            report_to="wandb",
        ),
        peft_config=lora_config,
        dataset_text_field=args.dataset_text_field,
    )

    # launch
    print("Training...")
    trainer.train()

    print("Saving the last checkpoint of the model")
    model.save_pretrained(os.path.join(args.output_dir, "final_checkpoint/"))
    if args.push_to_hub:
        trainer.push_to_hub("Upload model")
    print("Training Done! 💥")

In [ ]:
set_seed(args.seed)
os.makedirs(args.output_dir, exist_ok=True)

logging.set_verbosity_error()

main(args)

trainable params: 151369728 || all params: 1591200768 || trainable%: 9.5129245186488


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

wandb: Currently logged in as: 21051167 (aratrika123). Use `wandb login --relogin` to force relogin


Training...


{'loss': 2.7724, 'grad_norm': 0.6304113268852234, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 2.8533, 'grad_norm': 1.5150936841964722, 'learning_rate': 3.8e-05, 'epoch': 0.01}
{'loss': 2.7215, 'grad_norm': 1.1011840105056763, 'learning_rate': 5.8e-05, 'epoch': 0.01}
{'loss': 2.0094, 'grad_norm': 0.4177374243736267, 'learning_rate': 7.6e-05, 'epoch': 0.02}
{'loss': 2.3829, 'grad_norm': 0.5729718208312988, 'learning_rate': 9.6e-05, 'epoch': 0.02}
{'loss': 1.8465, 'grad_norm': 1.5647789239883423, 'learning_rate': 0.000116, 'epoch': 0.02}
{'loss': 1.7438, 'grad_norm': 0.7338495254516602, 'learning_rate': 0.00013600000000000003, 'epoch': 0.03}
{'loss': 1.5336, 'grad_norm': 0.6762092709541321, 'learning_rate': 0.00015600000000000002, 'epoch': 0.03}
{'loss': 1.4234, 'grad_norm': 0.9926688075065613, 'learning_rate': 0.00017600000000000002, 'epoch': 0.04}
{'loss': 1.2354, 'grad_norm': 1.0626448392868042, 'learning_rate': 0.000196, 'epoch': 0.04}
{'loss': 1.2884, 'grad_norm': 1.41360282897949

model.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

Training Done! 💥
